In [ ]:
f = open('Results.txt', 'r')
f.readline().split(' ')

In [ ]:
[int(el) for el in fansrd.readline().split(' ')]

In [1]:
import numpy as np
import pandas as pd

In [2]:
from encoders import polar_encode, polar_encode_crc
from decoders import polar_decode_sc1, polar_decode_sclist, polar_decode_scs
from decoders import polar_decode_sclist_crc, polar_decode_scs_crc

In [3]:
def bpsk_awgn(x, noiseVar, N):
    x_bpsk = 2 * x - 1
    rc_bpsk = x_bpsk + np.random.normal(0, np.sqrt(noiseVar), N)
    llrs = -4 * rc_bpsk / noiseVar
    return llrs

In [4]:
def run_sc(noiseVar, K, N, frozen_ind, poly, min_Itr=200, fe=20):
    f_sc = 0
    Itr = 0
    while f_sc < fe or Itr < min_Itr:
        u_K = np.random.randint(0, 2, size=K)
        [x, u] = polar_encode_crc(u_K, frozen_ind, N, poly)
        llrs = bpsk_awgn(x, noiseVar, N)
        u_sc, _, _ = polar_decode_sc1(llrs, frozen_ind, [])
        a = u == u_sc
        if a.sum() != N:
            f_sc += 1
        Itr += 1
    return f_sc/Itr, Itr

In [5]:
def run_sclist_crc(noiseVar, K, N, frozen_ind, poly, L, min_Itr=200, fe=20):
    f_sclist = 0
    Itr = 0
    while f_sclist < fe or Itr < min_Itr:
        u_K = np.random.randint(0, 2, size=K)
        [x, u] = polar_encode_crc(u_K, frozen_ind, N, poly)
        llrs = bpsk_awgn(x, noiseVar, N)
        u_sclist = polar_decode_sclist_crc(llrs, L, frozen_ind, poly)
        a = u == u_sclist
        if a.sum() != N:
            f_sclist += 1
        Itr += 1
    return f_sclist/Itr, Itr

In [6]:
def run_scstack_crc(noiseVar, K, N, n, frozen_ind, poly, L, D, min_Itr=200, fe=20):
    f_scstack = 0
    Itr = 0
    while f_scstack < fe or Itr < min_Itr:
        u_K = np.random.randint(0, 2, size=K)
        [x, u] = polar_encode_crc(u_K, frozen_ind, N, poly)
        llrs = bpsk_awgn(x, noiseVar, N)
        u_scstack = polar_decode_scs_crc(n, L, D, llrs, frozen_ind, poly)
        a = u == u_scstack
        if a.sum() != N:
            f_scstack += 1
        Itr += 1
    return f_scstack/Itr, Itr

In [ ]:
%timeit



In [7]:
poly = [1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]
l_poly = 11
K = 21
N = 64
n = 6

In [5]:
rank = pd.read_csv('rank.csv', sep = ' ', index_col=0)
rank = rank.Q.values

In [4]:
frozen_ind = np.zeros(N)
frozen_ind[rank[rank<N][:N-K-l_poly]] = 1

NameError: name 'N' is not defined

In [10]:
L_Array = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512] #, 1024, 2048, 4096]

In [13]:
data = []

In [12]:
EbNoArray = np.arange(0, 5, 0.2)
for EbNo in EbNoArray:
    noiseVar = 10**(-EbNo/10)
    fer, Itr = run_sc(noiseVar, K, N, frozen_ind, poly)
    data.append([EbNo, fer])
    print(np.round(EbNo, 2), fer, Itr)

0.0 0.595 200
0.2 0.495 200
0.4 0.47 200
0.6 0.44 200
0.8 0.42 200
1.0 0.4 200
1.2 0.28 200
1.4 0.265 200
1.6 0.265 200
1.8 0.15 200
2.0 0.15 200
2.2 0.16 200
2.4 0.11 200
2.6 0.11 200
2.8 0.042735042735042736 468
3.0 0.04484304932735426 446
3.2 0.033003300330033 606
3.4 0.02656042496679947 753
3.6 0.020703933747412008 966
3.8 0.012055455093429777 1659
4.0 0.007385524372230428 2708
4.2 0.004710315591144607 4246
4.4 0.003601656762110571 5553
4.6 0.0024084778420038534 8304
4.8 0.0010696903246510136 18697


In [14]:
EbNoArray = np.arange(0, 5, 0.2)
for i, EbNo in enumerate(EbNoArray):
    for L in L_Array:
        noiseVar = 10**(-EbNo/10)
        fer, Itr = run_sclist_crc(noiseVar, K, N, frozen_ind, poly, L)
        data.append(fer)
        print(np.round(EbNo, 2), L, fer, Itr)

0.0 1 0.615 200
0.0 2 0.41 200
0.0 4 0.325 200
0.0 8 0.205 200
0.0 16 0.23 200
0.0 32 0.12 200
0.0 64 0.105 200
0.0 128 0.05449591280653951 367
0.0 256 0.02656042496679947 753
0.0 512 0.029069767441860465 688
0.2 1 0.54 200
0.2 2 0.415 200
0.2 4 0.275 200
0.2 8 0.2 200
0.2 16 0.13 200
0.2 32 0.145 200
0.2 64 0.05141388174807198 389
0.2 128 0.04597701149425287 435
0.2 256 0.06006006006006006 333
0.2 512 0.019175455417066157 1043
0.4 1 0.51 200
0.4 2 0.315 200
0.4 4 0.215 200
0.4 8 0.13 200
0.4 16 0.0975609756097561 205
0.4 32 0.056338028169014086 355
0.4 64 0.03773584905660377 530
0.4 128 0.02557544757033248 782
0.4 256 0.020242914979757085 988
0.4 512 0.019175455417066157 1043
0.6 1 0.41 200
0.6 2 0.305 200
0.6 4 0.17 200
0.6 8 0.09433962264150944 212
0.6 16 0.0664451827242525 301
0.6 32 0.05115089514066496 391
0.6 64 0.032948929159802305 607
0.6 128 0.016353229762878167 1223
0.6 256 0.01697792869269949 1178
0.6 512 0.013717421124828532 1458
0.8 1 0.375 200
0.8 2 0.195 200
0.8 4 0.14 2

KeyboardInterrupt: 

In [12]:
col_scs = []
EbNoArray = np.arange(0.2, 5, 0.2)
D = 64
for i, EbNo in enumerate(EbNoArray):
    col_scs_l = []
    for L in L_Array:
        noiseVar = 10**(-EbNo/10)
        fer, Itr = run_scstack_crc(noiseVar, K, N, n, frozen_ind, poly, L, D)
        col_scs_l.append(fer)
        print(np.round(EbNo, 2), L, D, fer, Itr)
    col_scs.append(col_scs_l)

0.2 1 64 0.56 200
0.2 2 64 0.395 200
0.2 4 64 0.27 200
0.2 8 64 0.195 200
0.2 16 64 0.15 200
0.2 32 64 0.056338028169014086 355
0.2 64 64 0.06493506493506493 308
0.2 128 64 0.05698005698005698 351
0.2 256 64 0.0408997955010225 489
0.2 512 64 0.05847953216374269 342
0.2 1024 64 0.055248618784530384 362
0.2 2048 64 0.04866180048661801 411
0.2 4096 64 0.050505050505050504 396
0.4 1 64 0.485 200
0.4 2 64 0.32 200
0.4 4 64 0.195 200
0.4 8 64 0.12 200
0.4 16 64 0.115 200
0.4 32 64 0.07633587786259542 262
0.4 64 64 0.062111801242236024 322
0.4 128 64 0.029197080291970802 685
0.4 256 64 0.030441400304414 657
0.4 512 64 0.03225806451612903 620
0.4 1024 64 0.03616636528028933 553
0.4 2048 64 0.030864197530864196 648
0.4 4096 64 0.029850746268656716 670
0.6 1 64 0.425 200
0.6 2 64 0.315 200
0.6 4 64 0.15 200
0.6 8 64 0.14 200
0.6 16 64 0.07407407407407407 270
0.6 32 64 0.06309148264984227 317
0.6 64 64 0.02358490566037736 848
0.6 128 64 0.01607717041800643 1244
0.6 256 64 0.023696682464454975 844

KeyboardInterrupt: 

In [25]:
col_scs_l

[0.555,
 0.375,
 0.24,
 0.2,
 0.19,
 0.1,
 0.06993006993006994,
 0.09345794392523364,
 0.08620689655172414,
 0.06389776357827476,
 0.06451612903225806]

In [ ]:
0.4 1 0.535 200
0.4 2 0.455 200
0.4 4 0.38 200
0.4 8 0.43 200
0.4 16 0.355 200
0.4 32 0.32 200
0.4 64 0.36 200
0.4 128 0.375 200
0.4 256 0.44 200
0.4 512 0.395 200
0.4 1024 0.405 200
0.4 2048 0.375 200
0.4 4096 0.4 200
0.6 1 0.455 200
0.6 2 0.37 200
0.6 4 0.33 200
0.6 8 0.305 200
0.6 16 0.295 200
0.6 32 0.4 200
0.6 64 0.33 200

In [33]:
pd.read_txt()

In [19]:
len(data)

97

In [60]:
temp = []
start = 0
for i in range(10, 101, 10):
    print(start, i)
    print(len(data[start: i]))
    temp.append(np.round(data[start: i], 5))
    start = i
# temp.append(np.round(data[start: i], 5))

0 10
10
10 20
10
20 30
10
30 40
10
40 50
10
50 60
10
60 70
10
70 80
10
80 90
10
90 100
7


In [61]:
temp[-1]

array([0.18   , 0.06061, 0.03215, 0.01723, 0.0072 , 0.00499, 0.00176])

In [58]:
df = pd.DataFrame(np.array(temp))


In [62]:
df.loc[9] = [0.18   , 0.06061, 0.03215, 0.01723, 0.0072 , 0.00499, 0.00176, None, None, None]

In [64]:
df.to_csv('numbers.csv')

In [51]:
for row in np.array(temp).T:
    print('\n'.join(map(str, row)))
    print('======')

0.615
0.54
0.51
0.41
0.375
0.31
0.26
0.225
0.215
0.41
0.415
0.315
0.305
0.195
0.19
0.135
0.125
0.08734
0.325
0.275
0.215
0.17
0.14
0.08197
0.08032
0.07194
0.06645
0.205
0.2
0.13
0.09434
0.08264
0.09132
0.03817
0.03831
0.02833
0.23
0.13
0.09756
0.06645
0.05
0.0339
0.0241
0.02433
0.01765
0.12
0.145
0.05634
0.05115
0.05222
0.02721
0.02463
0.00887
0.00521
0.105
0.05141
0.03774
0.03295
0.0172
0.01103
0.00837
0.00705
0.00429
0.0545
0.04598
0.02558
0.01635
0.01656
0.0123
0.00466
0.00306
0.00166
0.02656
0.06006
0.02024
0.01698
0.00607
0.00953
0.0045
0.0019
0.00105
0.02907
0.01918
0.01918
0.01372
0.01095
0.00267
0.00329
0.0017
0.00062


In [36]:
np.array(temp).T.shape

(10, 9)

In [ ]:
K = 4
N = 8
n = 3

In [ ]:
EbNoArray = np.arange(0, 5, 0.2)
for i, EbNo in enumerate(EbNoArray):
    for L in L_Array:
        noiseVar = 10**(-EbNo/10)
        fer, Itr = run_sclist_crc(noiseVar, K, N, frozen_ind, poly, L)
        data.append(fer)
        print(np.round(EbNo, 2), L, fer, Itr)

In [21]:
poly = [1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]
l_poly = 11
K = 21
N = 64
n = 6

In [22]:
frozen_ind = np.zeros(N)
frozen_ind[rank[rank<N][:N-K-l_poly]] = 1

In [23]:
L = 8
D = 64

In [27]:
f_scstack = 0

u_K = np.random.randint(0, 2, size=K)
[x, u] = polar_encode_crc(u_K, frozen_ind, N, poly)
EbNo = 0.5
noiseVar = 10**(-EbNo/10)
llrs = bpsk_awgn(x, noiseVar, N)
u_scstack = polar_decode_scs_crc(n, L, D, np.round(llrs, 1), frozen_ind, poly)
u_sc, _, x_sc = polar_decode_sc1(np.round(llrs, 1), frozen_ind, [])
print('=======')
print(u, u_scstack)
print(u_sc, x)
a = u == u_scstack
if a.sum() != N:
    f_scstack += 1


[[0 0 list([])]] array len [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] stack len 1 max_metric 0
[[0.1 1 list([0])]] array len [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] stack len 1 max_metric 0.1
[[0.1 2 list([0, 0])]] array len [1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] stack len 1 max_metric 0.1
[[0.4 3 list([0, 0, 0])]] array len [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] stack len 1

  list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0])]] array len [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 3.
 4. 4. 4. 4. 4. 6. 6. 6. 6. 6. 5. 5. 3. 3. 3. 3. 3. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1.] stack len 64 max_metric 66.9
after first [[20.300000000000004 56
  list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1])]
 [20.599999999999998 41
  list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])]
 [21.1 28
  list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])]
 [21.200000000000003 23
  list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1])]
 [21.4 24
 

In [25]:
frozen_ind

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [28]:
f_scstack

0